In [ ]:
!nvidia-smi

Mon Apr 26 04:22:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
import matplotlib.pyplot as plt

import torch
#from torchtext import data
from torchtext.legacy import data
from torchtext import datasets

In [ ]:
device = torch.device(
  'cuda' if torch.cuda.is_available() else 'cpu'
)

In [ ]:
!pip install wget
import wget

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=49cbc927789ddddbf8998ef80fde76ca036387cb113e83056eb1685851566561
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
wget.download('https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar')

'dakshina_dataset_v1.0.tar'

In [ ]:
!tar -xvf  '/content/dakshina_dataset_v1.0.tar' -C '/content'

In [ ]:
# Training using "tamil" language dataset
# clone dakshina_dataset_v1.0/ta/lexicons/
# Folders with training, test data:
"""
dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.train.tsv
dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.dev.tsv  - WHAT IS THIS???
dakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.test.tsv
"""

'\ndakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.train.tsv\ndakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.dev.tsv  - WHAT IS THIS???\ndakshina_dataset_v1.0/ta/lexicons/ta.translit.sampled.test.tsv\n'

In [ ]:
# create Field objects
tamil_words = data.Field()
latin_words = data.Field()

# create tuples representing the columns
fields = [
  ('tamil_words', tamil_words),
  ('latin_words', latin_words),
  (None, None), # ignore last column
]

In [ ]:
train_data, valid_data, test_data = data.TabularDataset.splits(
   path = '/content/dakshina_dataset_v1.0/ta/lexicons',
   train = 'ta.translit.sampled.train.tsv',
   validation = 'ta.translit.sampled.dev.tsv',
   test = 'ta.translit.sampled.test.tsv',
   format = 'tsv',
   fields = fields,
   skip_header = True
)

In [ ]:
import random
#print(vars(train_data[11]))
for ii in range(10):
  a = random.randint(0, 100)
  print(vars(train_data[a]))

{'tamil_words': ['அகற்றக்'], 'latin_words': ['agattrak']}
{'tamil_words': ['அக'], 'latin_words': ['aka']}
{'tamil_words': ['அகச்சிவப்பு'], 'latin_words': ['akachchivappu']}
{'tamil_words': ['அக'], 'latin_words': ['aka']}
{'tamil_words': ['அகப்பொருள்'], 'latin_words': ['agapporul']}
{'tamil_words': ['அகற்றப்பட'], 'latin_words': ['akatrtrapada']}
{'tamil_words': ['அகதியாக'], 'latin_words': ['akathiyaaka']}
{'tamil_words': ['அகன்ற'], 'latin_words': ['agantra']}
{'tamil_words': ['ஃபோர்ஸ்'], 'latin_words': ['phoars']}
{'tamil_words': ['ஃபேஸ்புக்'], 'latin_words': ['paespuk']}


In [ ]:
tamil_words.build_vocab(train_data)
latin_words.build_vocab(train_data)
#print(len(tamil_words))

NameError: ignored

In [ ]:
# create iterators for train/valid/test datasets
train_loader, valid_loader, test_loader = data.BucketIterator.splits(
  (train_data, valid_data, test_data),
  sort_key = lambda x: x.tamil_words,
  sort = True,
  batch_size = 32,
  device = device
)

In [ ]:
class Encoder_RNN(nn.Module):
   def __init__(self, input_size, hidden_size, embbed_size, layers_n,model,dropout_prob):
       super(Encoder_RNN, self).__init__()
      
       self.input_size = input_size
       self.embbed_size = embbed_size
       self.hidden_size = hidden_size
       self.layers_n = layers_n
       self.model = model
       self.dropout_prob = dropout_prob

       #initialize the embedding layer with input and embbed dimention
       self.embedding = nn.Embedding(input_size, self.embbed_size)
       
       if model == "LSTM":
          self.lstm = nn.LSTM(self.embbed_size, self.hidden_size, layers_n=self.layers_n, batch_first=True)
       
       elif model == "GRU":
          self.gru = nn.GRU(self.embbed_size, self.hidden_size, layers_n=self.layers_n)

       self.dropout = nn.Dropout(dropout_prob)

   def forward(self, input):
      
       embedded = self.embedding(input).view(1,1,-1)
       if model == "LSTM":
          outputs, hidden = self.gru(embedded)
       elif model == "GRU":
          outputs, hidden = self.lstm(embedded)

       return outputs, hidden

In [ ]:
class Decoder_RNN(nn.Module):
   def __init__(self, output_dim, hidden_size, embbed_size, layers_n,model,dropout_prob):
       super(Decoder_RNN, self).__init__()

       self.embbed_size = embbed_size
       self.hidden_size = hidden_size
       self.output_dim = output_dim
       self.layers_n = layers_n
       self.model = model
       self.dropout_prob = dropout_prob

       self.embedding = nn.Embedding(output_dim, self.embbed_size)

       if model == "LSTM":
           self.lstm = nn.LSTM(self.embbed_size, self.hidden_size, layers_n=self.layers_n, batch_first=True)
       elif model == "GRU":
            self.gru = nn.GRU(self.embbed_size, self.hidden_size, layers_n=self.layers_n)
       
       self.out = nn.Linear(self.hidden_size, output_dim)
       self.softmax = nn.LogSoftmax(dim=1)

       self.dropout = nn.Dropout(dropout_prob)
      
   def forward(self, input, hidden):

       input = input.view(1, -1)
       embedded = F.relu(self.embedding(input))
       if model == "LSTM":
           output, hidden = self.lstm(embedded, hidden)                    
       elif model == "GRU":
           output, hidden = self.gru(embedded, hidden)       
       prediction = self.softmax(self.out(output[0]))
    
       return prediction, hidden